In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Cross Modal Retrieval'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Cross Modal Retrieval


In [2]:
!pip install open3d
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

     |████████████████████████████████| 4.6MB 74kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 21.6MB 1.3MB/s 
     |████████████████████████████████| 12.3MB 68.0MB/s 
     |████████████████████████████████| 18.2MB 147kB/s 
     |████████████████████████████████| 235kB 8.8MB/s 
     |████████████████████████████████| 2.1MB 12.9MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.5.0-cp36-none-any.whl size=267918 sha256=66799aecc851346817875b42111174b930c5bd4adf1bda535ba9b34c38673655
  Stored in directory: /root/.cache/pip/wheels/ec/51/31/5786f2ac419ee312f22d4d2877da05f20e7f2d430e22917daf
Successfully built torch-geometric


In [3]:
from text_extraction_torch import get_text_embedding
import open3d as o3d
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

embedding_matrix, encoder_input_sentences,modelIdList,categoryList = get_text_embedding("descriptions.csv")
print("embedding_matrix:",embedding_matrix.shape,"sentences:",encoder_input_sentences.shape,"modelIdList:",len(modelIdList))

embedding_matrix: (87, 200) sentences: (15, 47) modelIdList: 15


In [4]:
class ShapeText(Dataset):
    def __init__(self, encoder_input_sentences,modelIdList, categoryList, train=True, transform=None):
        self.is_train = train
        self.transform = transform
        self.encoder_input_sentences = encoder_input_sentences
        self.files = []
        for idx,modelId in enumerate(modelIdList):
            sample = os.path.join(categoryList[idx], modelId + ".ply")
            self.files.append(sample)
          
    
    def __len__(self):
        return len(self.files)
    
    def __preproc__(self, pcd_load):
        # convert Open3D.o3d.geometry.PointCloud to numpy array
        xyz_load = np.asarray(pcd_load.points)
        color_load = np.asarray(pcd_load.colors)
        if self.transform:
            xyz = self.transform(xyz_load)
        return xyz,color_load

    def __getitem__(self, idx):
        pcd_path = self.files[idx]
        pcd_load = o3d.io.read_point_cloud(pcd_path)
        pos_xyz, pos_color = self.__preproc__(pcd_load)
        pos_desc = self.encoder_input_sentences[idx]
        if not self.is_train:
          return self.files[idx], {'xyz': pos_xyz, 'color': pos_color}, pos_desc
        negative_idx = random.choices([ i for i in range(len(self.files)) if (i != idx and modelIdList[i]!=modelIdList[idx])],k=5)
        neg_shape = []
        neg_text = []
        for i in range(len(negative_idx)):
          neg_desc = self.encoder_input_sentences[i]
          pcd_path = self.files[i]
          pcd_load = o3d.io.read_point_cloud(pcd_path)
          neg_xyz, neg_color = self.__preproc__(pcd_load)
          neg_shape.append({'xyz': neg_xyz, 'color': neg_color})
          neg_text.append(neg_desc)
        return self.files[idx],{'xyz': pos_xyz, 'color': pos_color},pos_desc, neg_text,neg_shape

In [5]:
from torch import nn
class TripletLoss(nn.Module):
    def __init__(self, margin= 0.6):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
       # x1_norm = x1.norm(p=2,dim=1)
      #  x2_norm = x2.norm(p=2,dim=1)
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, positive_shape: torch.Tensor, positive_text: torch.Tensor, negative_text: torch.Tensor, negative_shape: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(positive_shape, positive_text)
        distance_negative = self.calc_euclidean(positive_shape, negative_text)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        distance_positive = self.calc_euclidean(positive_shape, positive_text)
        distance_negative = self.calc_euclidean(negative_shape, positive_text)
        losses += torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [6]:
import open3d as o3d
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud
        
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        return torch.from_numpy(pointcloud)
    
def default_transforms():
    return transforms.Compose([ Normalize(),
                                ToTensor()
                              ])

In [7]:
def calculateNearest(firstEmbed,secEmbed):
  correctIndex = []
  predictedIndex = []
  difference = []
  firstEmbed_norm = firstEmbed.norm(p=2,dim=1)
  secEmbed_norm = secEmbed.norm(p=2,dim=1)
  for i in range(len(firstEmbed)):
      distance = (secEmbed-firstEmbed[i]).pow(2).sum(1)
      idx, val= torch.argmin(distance).item(),torch.min(distance).item()
      correctIndex.append(i)
      predictedIndex.append(idx)
      difference.append(val)
  print("correctIndex:", correctIndex)
  print("predictedIndex:", predictedIndex)
  print("difference:", difference)
  return correctIndex,predictedIndex,difference

In [ ]:
from PointNet2 import Net
from embedding import Encoder
import random

train_split =  int(0.8 * len(modelIdList))
val_split = int(0.9 * len(modelIdList))
train_ds = ShapeText(encoder_input_sentences[ :train_split,:], modelIdList[ :train_split], categoryList[ :train_split], transform=default_transforms())

val_ds = ShapeText(encoder_input_sentences[train_split:val_split,:], modelIdList[train_split:val_split],
                     categoryList[train_split:val_split], train = False, transform=default_transforms())
test_ds = ShapeText(encoder_input_sentences[val_split: ,:], modelIdList[val_split: ], categoryList[val_split: ],
                    train = False, transform=default_transforms())
train_loader = DataLoader(dataset=train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_ds, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
HIDDEN_SIZE = 300
criterion = TripletLoss()
shape_model = Net(300).to(device)
text_model = Encoder(embedding_matrix, HIDDEN_SIZE, n_layers=2).to(device)
#shape_model.load_state_dict(torch.load("shape_model1.checkpoint"))
#text_model.load_state_dict(torch.load("text_model1.checkpoint"))
params = list(shape_model.parameters()) + list(text_model.parameters()) 
optimizer = torch.optim.Adam(params, lr=0.001)
shape_model.train()
text_model.train()
train_loss_history = []
train_acc_shape_history = []
train_acc_text_history = []
val_acc_shape_history = []
val_acc_text_history = []
val_loss_history = []
num_epochs = 5
iter_per_epoch = len(train_loader)
log_nth = 10
for epoch in range(num_epochs):
    running_loss = []
    total = 0
    corrText = 0
    corrShape = 0
    for i, (path, anchor_shape, pos_text, neg_text_list,neg_shape_list) in enumerate(train_loader):
      pos_text = pos_text.to(device)
      for idx in range(5):
        positive_out = text_model(pos_text[:,1:])
        anchor_out = shape_model(anchor_shape)
        neg_text = neg_text_list[idx].to(device)
        optimizer.zero_grad()
        neg_shape_out = shape_model(neg_shape_list[idx])
        negative_out = text_model(neg_text[:,1:])
        loss = criterion(anchor_out, positive_out, negative_out, neg_shape_out)
        loss.backward()
        optimizer.step()
        
      
      if (i+1) % log_nth == 0:
          
          print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch+1, num_epochs, i+1, iter_per_epoch, loss.item()))
          
      if  (i+1)%500 == 0 : 
          print("pos_out:",positive_out[0,:10],"neg_out:",negative_out[0,:10],"anchor_out:",anchor_out[0,:10])
          correctText,predictedText,difference = calculateNearest(anchor_out, positive_out)
          correctShape,predictedShape,difference = calculateNearest(positive_out, anchor_out)
          total = len(correctShape)
          corrText = np.sum(np.array(correctText) == np.array(predictedText))
          corrShape = np.sum(np.array(correctShape) == np.array(predictedShape))
          shapeAcc = 100 * (corrShape / total)
          textAcc = 100 * (corrText / total)
          print('Train: Text to Shape Retrieval accuracy: {} %'.format(shapeAcc) )
          print('Train: Shape to Text Retrieval accuracy: {} %'.format(textAcc) )
          train_acc_shape_history.append(shapeAcc)
          train_acc_text_history.append(textAcc)
      # Validate the model
      if  (i+1)%iter_per_epoch == 0:
        shape_model.eval()
        text_model.eval()
        torch.save(shape_model.state_dict(), "shape_model_triplet.checkpoint")
        torch.save(text_model.state_dict(), "text_model_triplet.checkpoint")
        with torch.no_grad():
          corrText = 0
          corrShape = 0
          total = 0
          for (path, shape, text) in val_loader:
              text = text.to(device)
              text_embed = text_model(text)
              shape_embed = shape_model(shape)
              correctText,predictedText,difference = calculateNearest(shape_embed, text_embed)
              correctShape,predictedShape,difference = calculateNearest(text_embed, shape_embed)
              total += len(correctShape)
              corrText += np.sum(np.array(correctText) == np.array(predictedText))
              corrShape += np.sum(np.array(correctShape) == np.array(predictedShape))

          shapeAcc = 100 * (corrShape / total)
          textAcc = 100 * (corrText / total)
          print('Validate: Text to Shape Retrieval accuracy: {} %'.format(shapeAcc) )
          print('Validate: Shape to Text Retrieval accuracy: {} %'.format(textAcc) )
          val_acc_shape_history.append(shapeAcc)
          val_acc_text_history.append(textAcc)
        shape_model.train()
        text_model.train()

In [ ]:
def calculateAllNearest(firstEmbed, secEmbed, path):
  rr_1 = []
  rr_5 = []
  rr_10 = []
  firstEmbed_norm = firstEmbed.norm(p=2,dim=1)
  secEmbed_norm = secEmbed.norm(p=2,dim=1)
  for i in range(len(firstEmbed)):
      distance = (secEmbed-firstEmbed[i]).pow(2).sum(1)
      val_10, idx_10 = torch.topk(distance, 10, largest = False)
      rr_1.append(path[i]) if idx_10[0] == i else rr_1.append("")
      rr_5.append(path[i]) if i in idx_10[:5] else rr_5.append("")
      rr_10.append(path[i]) if i in idx_10 else rr_10.append("")
  print("func:",len(rr_1),len(rr_5),len(rr_10))
  return rr_1,rr_5,rr_10

  
def write_file(filepath,rr1_Text,rr5_Text,rr10_Text,rr1_Shape,rr5_Shape,rr10_Shape):
  print("file written")
  with open(filepath, 'w') as file:
    file.write("RR@1 Text Accuracy:\n")
    for s in rr1_Text:
      if s != "":
        file.write(s)
        file.write("\n")
    file.write("RR@5 Text Accuracy:\n")
    for s in rr5_Text:
      if s != "":
        file.write(s) 
        file.write("\n")
    file.write("RR@10 Text Accuracy:\n")
    for s in rr10_Text:
      if s != "":
        file.write(s)
        file.write("\n")
    file.write("RR@1 Shape Accuracy:\n")
    for s in rr1_Shape:
      if s != "":
        file.write(s)
        file.write("\n")
    file.write("RR@5 Shape Accuracy:\n")
    for s in rr5_Shape:
      if s != "":
        file.write(s)
        file.write("\n")
    file.write("RR@10 Shape Accuracy:\n")
    for s in rr10_Shape:
      if s != "":
        file.write(s) 
        file.write("\n")
    file.close()

In [ ]:
from PointNet2 import Net
from embedding import Encoder
from embedding import ShapeDecoder
from TextDecoder import Decoder
import random
train_split =  int(0.9 * len(modelIdList))
val_split = int(0.95 * len(modelIdList))

#val_ds = ShapeText(encoder_input_sentences[train_split:val_split,:], modelIdList[train_split:val_split],
#                     categoryList[train_split:val_split], train = False, transform=default_transforms())
test_ds = ShapeText(encoder_input_sentences[val_split: ,:], modelIdList[val_split: ], categoryList[val_split: ],
                    train = False, transform=default_transforms())
#val_loader = DataLoader(dataset=val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_ds, batch_size=64, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
HIDDEN_SIZE = 300
shape_model = Net(300).to(device)
text_model = Encoder(embedding_matrix, HIDDEN_SIZE, n_layers=2).to(device)
shape_decoder = ShapeDecoder(300).to(device)
text_decoder = Decoder(embedding_matrix.astype(np.float32), embedding_matrix.shape[0], 300).to(device)
shape_model.load_state_dict(torch.load("shape_model_triplet.checkpoint"))
text_model.load_state_dict(torch.load("text_model_triplet.checkpoint"))
shape_embed_list = torch.zeros(0, 0).to(device)
text_embed_list = torch.zeros(0, 0).to(device)
path_list=[]
print(len(modelIdList))
with torch.no_grad():
    corrText = 0
    corrShape = 0
    total = 0
    for path,shape, text in test_loader:
      text = text.to(device)
      text_embed = text_model(text)
      shape_embed = shape_model(shape)
      text_embed_list = torch.cat((text_embed_list,text_embed),dim=0) if len(text_embed_list)!=0 else text_embed
      shape_embed_list = torch.cat((shape_embed_list,shape_embed),dim=0) if len(shape_embed_list)!=0 else shape_embed
      path_list.extend(list(path))
      print(len(path_list))
    rr1_Text,rr5_Text,rr10_Text = calculateAllNearest(shape_embed_list, text_embed_list, path_list)
    rr1_Shape,rr5_Shape,rr10_Shape = calculateAllNearest(text_embed_list, shape_embed_list, path_list)
    rr1TextAcc,rr5TextAcc, rr10TextAcc = np.sum(np.array(rr1_Text)!=""),np.sum(np.array(rr5_Text)!=""),np.sum(np.array(rr10_Text)!="")
    rr1ShapeAcc,rr5ShapeAcc, rr10ShapeAcc = np.sum(np.array(rr1_Shape)!=""),np.sum(np.array(rr5_Shape)!=""),np.sum(np.array(rr10_Shape)!="")
    total = 100/len(rr1_Shape)
    write_file("triplet_loss_results",rr1_Text,rr5_Text,rr10_Text,rr1_Shape,rr5_Shape,rr10_Shape)
    print("rr1ShapeAcc:", rr1ShapeAcc*total, "rr5ShapeAcc:", rr5ShapeAcc*total, "rr10ShapeAcc:", rr10ShapeAcc*total)
    print("rr1TextAcc:", rr1TextAcc*total, "rr5TextAcc:", rr5TextAcc*total, "rr10TextAcc:", rr10TextAcc*total)
      

72759
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3638
func: 3638 3638 3638
func: 3638 3638 3638
file written
rr1ShapeAcc: 0.35733919736118747 rr5ShapeAcc: 1.6767454645409565 rr10ShapeAcc: 2.9686641011544803
rr1TextAcc: 0.4123144584936778 rr5TextAcc: 1.456844420010995 rr10TextAcc: 2.3639362286970864


In [17]:
x=torch.randn(2,3,2)
print(x)
val,idx =torch.max(x,dim=1)
val.shape

tensor([[[ 0.6127, -1.0223],
         [ 1.6285, -1.9668],
         [-0.9557, -0.4103]],

        [[-1.4859, -0.5730],
         [ 0.0153, -0.3187],
         [-1.6082,  1.0959]]])


torch.Size([2, 2])